# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
# Скачиваем данные

!wget https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv

--2022-09-01 11:54:29--  https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv
Распознаётся data.gov.ru (data.gov.ru)… 212.164.137.119
Подключение к data.gov.ru (data.gov.ru)|212.164.137.119|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1204 (1,2K) [text/plain]
Сохранение в: «data-20160608T1215-structure-20160608T1215.csv»

data-20160608T1215- 100%[===================>]   1,18K  --.-KB/s    за 0s      

2022-09-01 11:54:30 (1,12 GB/s) - «data-20160608T1215-structure-20160608T1215.csv» сохранён [1204/1204]



In [3]:
data = pd.read_csv('data-20160608T1215-structure-20160608T1215.csv', index_col='Показатель', sep=',')
data

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [4]:
# С помощью регулярных выражений вделяем минимальное и максимальное допустимое значение норматива

data['Норматив'] = data['Норматив'].replace(to_replace=r'(не более (\d-)?)', value='0<', regex=True)
data['Норматив'] = data['Норматив'].replace(to_replace=r'в пределах ', value='', regex=True)
data['Норматив'] = data['Норматив'].replace(to_replace=r',', value='.', regex=True)
data['Норматив'] = data['Норматив'].replace(to_replace=r'-', value='<', regex=True)
data_temp = data['Норматив'].str.split('<',expand=True)
data_temp

,0,1
Показатель,,
pH,6,9
Запах,0,3
Цветность,0,30
Жёсткость,7,10
Аммиак и аммоний-ион (по азоту),0,1.5
Нитриты (по NO2),0,3.3
Нитраты (по NO3),0,45
Фосфаты (P),0,3.5
Хлориды (Cl),0,350


In [5]:
data = pd.concat([data, data_temp], axis=1)
data.rename(columns={0: 'Минимальное значение', 1: 'Максимальное значение'}, inplace=True)

In [6]:
data.drop(columns=['Норматив'], inplace=True)
data

,Единица измерений,Результат анализа,Минимальное значение,Максимальное значение
Показатель,,,,
pH,единицы pH,8.4,6,9
Запах,баллы,1,0,3
Цветность,градусы,б/цвета,0,30
Жёсткость,мг-эквл/дм3,9.199999999999999,7,10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,0,1.5
Нитриты (по NO2),мг/дм3,0.017,0,3.3
Нитраты (по NO3),мг/дм3,24,0,45
Фосфаты (P),мг/дм3,0.36,0,3.5
Хлориды (Cl),мг/дм3,200,0,350


In [7]:
# Будем считать результат б/цвета за 0

data['Результат анализа'] = data['Результат анализа'].replace(to_replace=r'б/цвета', value='0', regex=True)

In [8]:
# Приведем тип данных к числовому 'float32'

data[['Результат анализа', 'Минимальное значение', 'Максимальное значение']] = data[
                                                                                ['Результат анализа',
                                                                                 'Минимальное значение',
                                                                                 'Максимальное значение']].astype('float32')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, pH to Окисляемость перманганатная
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Единица измерений      14 non-null     object 
 1   Результат анализа      14 non-null     float32
 2   Минимальное значение   14 non-null     float32
 3   Максимальное значение  14 non-null     float32
dtypes: float32(3), object(1)
memory usage: 392.0+ bytes


In [10]:
# С помощью запросов определим соотвествие измерений нормативу

data_good = data.query('`Минимальное значение` <= `Результат анализа` <= `Максимальное значение`')
data_bad = data.query('`Результат анализа` > `Максимальное значение`')

data_good['Результат'] = 'Удовлетворительно'
data_bad['Результат'] = 'Неудовлетворительно'

data_final = pd.concat([data_bad, data_good], axis=0)
data_final

/var/folders/rw/fy4jnmhn3l72v47ky8bt9r9w0000gn/T/ipykernel_32770/1566767372.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_good['Результат'] = 'Удовлетворительно'
/var/folders/rw/fy4jnmhn3l72v47ky8bt9r9w0000gn/T/ipykernel_32770/1566767372.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bad['Результат'] = 'Неудовлетворительно'


,Единица измерений,Результат анализа,Минимальное значение,Максимальное значение,Результат
Показатель,,,,,
Нефть,мг/дм3,0.550,0.0,0.3,Неудовлетворительно
Окисляемость перманганатная,мг/дм3,2.000,0.0,0.5,Неудовлетворительно
pH,единицы pH,8.400,6.0,9.0,Удовлетворительно
Запах,баллы,1.000,0.0,3.0,Удовлетворительно
Цветность,градусы,0.000,0.0,30.0,Удовлетворительно
Жёсткость,мг-эквл/дм3,9.200,7.0,10.0,Удовлетворительно
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,0.0,1.5,Удовлетворительно
Нитриты (по NO2),мг/дм3,0.017,0.0,3.3,Удовлетворительно
Нитраты (по NO3),мг/дм3,24.000,0.0,45.0,Удовлетворительно


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [11]:
def fruit(quantity=3, number_test=1000000, q_orange=5, q_apple=4):
    result = []
    for test in range(1, number_test + 1):
        hand = []
        basket = ['orange']*q_orange + ['apple']*q_apple
        for i in range(quantity):
            fruit = random.choice(basket)
            hand.append(fruit)
            basket.remove(fruit)
        result.append(hand)
    return(result)
    


In [12]:
data_fruit = pd.DataFrame(fruit(), columns=['A','B','C'])
probablity = len(data_fruit.query('A == B == C == "orange"')) / len(data_fruit)
print('Вероятность выпадения трех апельсинов подряд равна ',round(probablity, 3))

Вероятность выпадения трех апельсинов подряд равна  0.119


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [13]:
def detail(quantity=2, number_test=1000000, q_standart=7, q_nonstandart=3):
    result = []
    for test in range(1, number_test + 1):
        master = []
        box = ['standart']*q_standart + ['nonstandart']*q_nonstandart
        for i in range(quantity):
            detail = random.choice(box)
            master.append(detail)
            box.remove(detail)
        result.append(master)
    return(result)
    

In [14]:
data_detail = pd.DataFrame(detail(), columns=['A','B'])
probablity = len(data_detail.query('A == "nonstandart" & B == "standart"')) / len(data_detail)
print('Вероятность того что мастер проверит 2 детали равна ',round(probablity, 3))

Вероятность того что мастер проверит 2 детали равна  0.233
